In [ ]:
import enum
from dataclasses import dataclass

import pandas as pd
from matplotlib import pyplot

class Action(enum.Enum):
    NONE = 0
    SELF_USE = 1,
    FEED_IN = 2,
    BACKUP = 3,
    CHARGE = 4

@dataclass
class TimeSegment:
    generation: float
    consumption: float
    feed_in_tariff: float
    import_tariff: float

@dataclass
class RunResult:
    import_cost: float = 0
    feed_in_cost: float = 0
    battery_level: float = 0

SEGMENT_LENGTH_SECS = 60 * 60
BATTERY_CAPACITY = 4.2
BATTERY_CHARGE_AMOUNT_PER_SEGMENT = 2 * (SEGMENT_LENGTH_SECS / (60 * 60))
EXPORT_LIMIT_PER_SEGMENT = 9999

def run(segments: list[TimeSegment], actions: list[Action], initial_battery: float) -> RunResult:
    action = Action.SELF_USE
    battery_level = initial_battery
    result = RunResult()

    # Debug
    battery_levels = []
    imports = []
    exports = []

    for segment, action_change in zip(segments, actions):
        if action_change != Action.NONE:
            action = action_change

        if action == Action.SELF_USE:
            # If generation can cover consumption, excess goes into battery. Else excess comes from battery if available
            if segment.generation > segment.consumption:
                excess_solar = segment.generation - segment.consumption
                solar_added_to_battery = min(BATTERY_CAPACITY - battery_level, excess_solar)
                battery_level += solar_added_to_battery
                result.feed_in_cost += min(EXPORT_LIMIT_PER_SEGMENT, excess_solar - solar_added_to_battery) * segment.feed_in_tariff
            
                imports.append(0)
                exports.append(min(EXPORT_LIMIT_PER_SEGMENT, excess_solar - solar_added_to_battery))
            else:
                required_energy = segment.consumption - segment.generation
                consumed_from_battery = min(battery_level, required_energy)
                battery_level -= consumed_from_battery
                result.import_cost += (required_energy - consumed_from_battery) * segment.import_tariff

                imports.append(required_energy - consumed_from_battery)
                exports.append(0)
        elif action == Action.FEED_IN:
            # Generation goes to grid rather than battery (up to export limit), but consumption still draws from battery
            if segment.generation > segment.consumption:
                excess_solar = segment.generation - segment.consumption
                solar_exported = min(EXPORT_LIMIT_PER_SEGMENT, excess_solar)
                result.feed_in_cost += solar_exported * segment.feed_in_tariff
                battery_level += min(BATTERY_CAPACITY, excess_solar - solar_exported)

                imports.append(0)
                exports.append(solar_exported)
            else:
                required_energy = segment.consumption - segment.generation
                consumed_from_battery = min(battery_level, required_energy)
                battery_level -= consumed_from_battery
                result.import_cost += (required_energy - consumed_from_battery) * segment.import_tariff

                imports.append(required_energy - consumed_from_battery)
                exports.append(0)
        elif action == Action.BACKUP:
            # PV goes first to batteries, then to house, with excess being exported
            solar_to_battery = min(BATTERY_CAPACITY - battery_level, segment.generation)
            battery_level += solar_to_battery
            excess_solar = segment.generation - solar_to_battery
            if excess_solar > segment.consumption:
                result.feed_in_cost += min(EXPORT_LIMIT_PER_SEGMENT, excess_solar - segment.consumption) * segment.feed_in_tariff

                imports.append(0)
                exports.append(min(EXPORT_LIMIT_PER_SEGMENT, excess_solar - segment.consumption))
            else:
                result.import_cost += (segment.consumption - excess_solar) * segment.import_tariff

                imports.append(segment.consumption - excess_solar)
                exports.append(0)
        elif action == Action.CHARGE:
            # I don't actually know, but... I assume that solar replaces grid up to the charge rate
            solar_to_battery = min(BATTERY_CAPACITY - battery_level, segment.generation)
            grid_to_battery = min(BATTERY_CAPACITY - battery_level - solar_to_battery, BATTERY_CHARGE_AMOUNT_PER_SEGMENT)
            battery_level += solar_to_battery + grid_to_battery
            excess_solar = segment.generation - solar_to_battery
            # Doesn't consider inverter limits
            result.import_cost += grid_to_battery * segment.import_tariff
            if excess_solar > segment.consumption:
                result.feed_in_cost += min(EXPORT_LIMIT_PER_SEGMENT, excess_solar - segment.consumption) * segment.feed_in_tariff
            
                imports.append(grid_to_battery)
                exports.append(min(EXPORT_LIMIT_PER_SEGMENT, excess_solar - segment.consumption))
            else:
                result.import_cost += (segment.consumption - excess_solar) * segment.import_tariff

                imports.append(grid_to_battery + segment.consumption - excess_solar)
                exports.append(0)

        battery_levels.append(battery_level)

    pyplot.plot(range(0, 24), battery_levels, marker='o', label='batt')
    pyplot.plot(range(0, 24), [x.consumption for x in segments], marker='x', label='cons')
    pyplot.plot(range(0, 24), [x.generation for x in segments], marker='+', label='gen')
    pyplot.plot(range(0, 24), imports, marker='<', label='gen')
    pyplot.plot(range(0, 24), exports, marker='>', label='gen')
    pyplot.show()

    result.battery_level = battery_level
    return result


# df = pd.read_csv('load_power_hourly.csv', usecols=['start', 'mean'], parse_dates=['start'], index_col='start') 
# consumption = [x for x in df[21:(21+24)]['mean']]
consumption = [0.2, 0.4, 0.2, 0.2, 0.2, 0.31, 0.25, 0.41, 0.32, 0.32, 0.29, 0.4, 0.57, 0.53, 0.82, 0.32, 0.32, 0.22, 0.11, 0.45, 0, 0, 0, 0]
generation = [0, 0, 0, 0, 0, 0, 0.05, 0.15, 0.72, 2.04, 2.33, 2.27, 2.35, 2.1, 1.94, 1.26, 0.75, 0.26, 0.11, 0.06, 0, 0, 0, 0]
import_tariff = [30.72] * 2 + [18.43] * 3 + [30.72] * 11 + [43.01] * 3 + [30.72] * 5
feed_in_tariff = [19.72] * 2 + [7.43] * 3 + [19.72] * 11 + [32.01] * 3 + [19.72] * 5

segments = [TimeSegment(generation=g, consumption=c, feed_in_tariff=f, import_tariff=i) for g, c, f, i in zip(generation, consumption, feed_in_tariff, import_tariff)]

actions = [Action.SELF_USE] * 2 + [Action.BACKUP] * 3 + [Action.SELF_USE] * 11 + [Action.FEED_IN] * 3 + [Action.SELF_USE] * 5

result = run(segments, actions, 2.1)
print(result)